In [45]:
class McDonald_Word_List:
    def __init__(self, pos_words, neg_words):
        self.pos_words = pos_words
        self.neg_words = neg_words
        self.pos_word_counts = {word:0 for word, val in pos_words.items()}
        self.intersection_pos = {word:0 for word, val in pos_words.items()}
        self.neg_word_counts = {word:0 for word, val in neg_words.items()}
        self.intersection_neg = {word:0 for word,val in neg_words.items()}
        
    def __str__(self):
        print("""Pos words: {0}
                 Neg words: {1}""".format(
                  len(self.pos_words),
                  len(self.neg_words)))

We will now read Bill McDonald's Excel file containing the master dictionary of financial sentiment words.
For this task, I am using the xlrd library. For now, I am only reading the cell values that have words with positive or negative sentiment.

In [46]:
from xlrd import open_workbook
FORMAT = ['Positive', 'Negative']
values = ""

wb = open_workbook('McDonaldDict.xlsx')

values = []
for s in wb.sheets():
    #print 'Sheet:',s.name
    words = []
    pos = {}
    neg = {}
    for row in range(1, s.nrows):
        col_names = s.row(0)[1:]
        col_value = []
        word = s.cell(row, 0).value
        for name, col in zip(col_names, range(1,s.ncols)):
            value = (s.cell(row,col).value)
            if name.value == 'Positive' and int(value) > 0:
                pos[word] = int(value)
            elif name.value == 'Negative' and int(value) > 0:
                neg[word] = int(value)
            col_value.append((name.value, value))
        values.append(col_value)
mcd = McDonald_Word_List(pos, neg)
print(mcd.pos_words)

{'COMPLIMENTED': 2009, 'ATTRACTIVE': 2009, 'SUCCEED': 2009, 'DISTINCTIVELY': 2009, 'PROSPERED': 2009, 'ACHIEVES': 2009, 'PROFICIENTLY': 2009, 'ENABLED': 2009, 'FAVORITES': 2009, 'PROACTIVE': 2009, 'INTEGRITY': 2009, 'ENCOURAGES': 2009, 'COLLABORATED': 2009, 'COMPLIMENTS': 2009, 'ENHANCEMENTS': 2009, 'COLLABORATIONS': 2009, 'FAVORABLE': 2009, 'PLEASURE': 2009, 'PROGRESS': 2009, 'EXCELLENT': 2009, 'GREATLY': 2009, 'PLENTIFUL': 2009, 'IMPROVE': 2009, 'REVOLUTIONIZES': 2009, 'ADVANCES': 2009, 'EMPOWERED': 2009, 'BREAKTHROUGH': 2009, 'ENTHUSIASTIC': 2009, 'ACHIEVE': 2009, 'EXCLUSIVELY': 2009, 'INNOVATIVENESS': 2011, 'PREEMINENCE': 2009, 'UPTURN': 2009, 'INCREDIBLE': 2009, 'IMPROVEMENT': 2009, 'BETTER': 2009, 'DELIGHTS': 2009, 'ENHANCING': 2009, 'UNPARALLELED': 2009, 'DELIGHTING': 2009, 'EMPOWERING': 2009, 'SUCCEEDING': 2009, 'EFFICIENCIES': 2009, 'HONORED': 2009, 'STRONG': 2009, 'WINNING': 2009, 'STABILITY': 2009, 'PRESTIGIOUS': 2009, 'COMPLIMENTARY': 2009, 'ASSURES': 2009, 'PRESTIGE': 2009

Voila. We have our lists of positive and negative words generated. These words were annotated for the financial domain and will be what we use to analyze our pointwise mutual information across the corpus.

In [5]:
import re
import sys

We will use the method below to extract header information from the news article. Additionally, we get rid of the miscellaneous header text in order to extract text more precisely.

In [63]:
def extract_header(text):
    try:
        print(text)
        sys.stdout.flush()
        search = re.search('--(.+?)--(.+?)--(.+?)--(.+?)Reuters\)\s-', text, flags=re.DOTALL)
        text = re.sub('--.+?--.+?--.+?--.+?Reuters\)\s-', '', text, flags=re.DOTALL)
        title = search.group(1)
        author = search.group(2)
        date = search.group(3)
        link = search.group(4)
        return title, author, date, link, text
    except Exception as e:
        return '','','','',''

In [69]:
import os

path = os.getcwd() 

reuters_folders = os.listdir('/home/jmkovachi/Documents/jupyter_notebooks/reuters')[1000:1010]

path += '/reuters'

articles = []

In [10]:
print(reuters_folders)

['20090106', '20071103', '20081209', '20090908', '20130619', '20110530', '20070328', '20121009', '20081101', '20130719']


In [70]:
count = 0
for folder in reuters_folders:
    article_files = os.listdir(path + '/' + folder)
    sys.stdout.flush()
    for file in article_files:
        count += 1
        with open(path + '/' + folder + '/' + file) as f:
            raw_text = f.read()
            title, author, date, link, text = extract_header(raw_text)
            articles.append({'title' : title, 'author' : author, 'date' : date, 'link' : link, 'text' : text})
            #print(articles[len(articles)-1]['text'] if count % 100 == 0 else '')
            



-- House Democrat says budget deal "very close"
-- 
-- Fri Apr 8, 2011 7:15am EDT
-- http://www.reuters.com/article/2011/04/08/us-usa-budget-hoyer-idUSTRE73725M20110408

 

 WASHINGTON  (Reuters) - House of Representatives Democratic Whip Steny Hoyer said on Friday that budget negotiators were "very close" to a deal to avoid a shutdown of the federal government. 

 "There's no deal yet unfortunately," Hoyer told NBC's Today show program as negotiators worked furiously to reach agreement. "I think we're very close. I think we've come 70 percent of the way in terms of dollars. That's a long way to go in trying to reach compromise." (Reporting by  David Morgan ; Editing by  Vicki Allen )
-- Scenarios: What could happen in budget fight
-- By  Richard Cowan
-- Fri Apr 8, 2011 7:47pm EDT
-- http://www.reuters.com/article/2011/04/08/us-usa-budget-scenarios-idUSTRE7378CF20110408

 

 WASHINGTON  (Reuters) - President Barack Obama and leaders in the U.S. Congress are racing against a midnight F

-- Special Report: Why London can live without its big banks
-- By  Paul Hoskins
-- Fri Apr 8, 2011 8:41am EDT
-- http://www.reuters.com/article/2011/04/08/us-britain-banks-idUSTRE73725G20110408

 

 LONDON  (Reuters) - "Let me be crystal clear," Barclays' star investment banker and new Chief Executive Bob Diamond told members of the British parliament in January. "We are going to be here in the United Kingdom, and this is the place that we want to succeed." 

 It may be a reflection of how bankers still struggle to be taken at their word in the wake of the global financial crisis, but Diamond's statement convinced nobody. Rumors that Barclays and its bigger rival HSBC are ready to move their headquarters to Hong Kong or New York have flourished ever since. Speculation has intensified in the run-up to the April 11 publication of an interim report into the future of British financial services by the Independent Commission on Banking (ICB). The banks fear it will recommend costly reforms

-- Google seals ITA deal; larger antitrust review looms
-- By  Diane Bartz
-- Fri Apr 8, 2011 5:12pm EDT
-- http://www.reuters.com/article/2011/04/08/us-ita-google-antitrust-idUSTRE7373DK20110408

 

 WASHINGTON  (Reuters) - The Justice Department approved Google Inc's purchase of ITA Software with stiff conditions on Friday, and left the door open to a larger probe into whether Google manipulates its search results to hurt rivals. 

 The Justice Department blessed the $700 million deal with the ticketing software company. Google promised to license the software for five years, to continue to upgrade it, and to establish firewalls to protect ITA clients' intellectual property. The ITA buy is part of an acquisition and hiring spree as Google aims to ensure its online services stay on top as Internet surfers go mobile and turn to services like the wildly popular Facebook. Now, U.S. antitrust regulators will turn to the question of whether to open a formal antitrust probe into allegations

-- HTC first quarter profit triples on Android popularity
-- 
-- Fri Apr 8, 2011 5:08am EDT
-- http://www.reuters.com/article/2011/04/08/us-htc-idUSTRE7371GE20110408

 

 TAIPEI  (Reuters) - Taiwan smartphone maker HTC Corp said first-quarter profit almost tripled, beating forecasts, driven by strong demand for its handsets, particularly those running on the Android operating system 

 The company, which has just overtaken erstwhile industry giant Nokia Oyj in terms of market capitalization, said on Friday that unaudited first-quarter net profit was T$14.83 billion ($511 million). Analysts had forecast a profit of T$12.99 billion. "That its first quarter would be above expectations was well foreseen, Q1 seasonality was better than expected," said Bonnie Chang, an analyst at Yuanta Securities in Hong Kong. "For the second quarter everyone is expecting revenue sequential growth in the high teens to 20 percent, shipments will be strong and average selling prices are holding up pretty well

-- SEC wrestling with private trading rules
-- By Sarah N. Lynch and  Clare Baldwin
-- Fri Apr 8, 2011 5:13pm EDT
-- http://www.reuters.com/article/2011/04/08/us-sec-schapiro-idUSTRE7373MC20110408

 

 WASHINGTON/NEW YORK  (Reuters) - U.S. securities regulators are conducting a broad review of share issuance rules for private companies, including the 500-shareholder threshold meant to mark the transition to public ownership. 

 The Securities and Exchange Commission review could make it easier for private companies to raise capital while delaying an initial public offering and the accompanying increase in financial disclosures. "We are approaching it with an open mind," SEC Chairman Mary Schapiro said of the policy review on Friday to a group of business journalists. "I really can't predict what, if any, change will come out of that." The issue has jumped into the spotlight recently as Wall Street banks and electronic markets offer investors a chance to buy and actively trade stakes in

...

 Dimon said the company is still paying the price for doing the Federal Reserve "a favor" by buying Bear Stearns in early 2008. "I'm going to say we've lost $5 billion to $10 billion on various things related to Bear Stearns now. And yes, I put it in the unfair category," Dimon said, speaking at a Council on Foreign Relations event in Washington. Dimon's candid comments come one week after the New York State Attorney General filed a lawsuit against JPMorgan, alleging that Bear Stearns deceived investors buying mortgage-backed securities in 2006 and 2007. During a wide-ranging hour-long discussion that went from the "fiscal cliff" to the impact of regulations, Dimon bristled when a member of the audience asked him if he now regretted participating with the government to rescue Bear Stearns in light of the lawsuit. "We didn't participate with the Federal Reserve, OK?" he said. "Let's get this one exactly right. We were asked to do it. We did it at great risk to ourselves ... Would I have

-- Stores see busy, but not bang-up Christmas Eve
-- By  Martinne Geller  and  Lisa Baertlein
-- Sun Dec 25, 2011 8:26am EST
-- http://www.reuters.com/article/2011/12/25/us-usa-retail-idUSTRE7BN0CS20111225

 

 NEW YORK/LAS VEGAS  (Reuters) - Retailers saw a steady flow of last-minute shoppers on Saturday, the day before Christmas, putting a moderate cap on a pre-holiday season that started with a bang and has since waned. 
 Industry watchers are forecasting a stronger holiday shopping season than expected, fueled by deep discounts at the start of the season, unusually warm and dry weather, a late Hanukkah, and an extra shopping day. On the last shopping day before Christmas, the scene at several malls in different parts of the country was busy, but neither shoppers nor retailers seemed overwhelmed. "The last-minute Charlies have come out," said Marshal Cohen, chief industry analyst at the NPD Group. "Stores are busy, but not bustling." The fact that Christmas Eve falls on a Saturday i

-- Japan looking to buy Chinese government bonds
-- 
-- Sun Dec 25, 2011 7:33am EST
-- http://www.reuters.com/article/2011/12/25/us-japan-china-bonds-idUSTRE7BO06G20111225

 

 BEIJING  (Reuters) - Japan will seek to buy Chinese government bonds, the Japanese government said in a statement issued after Prime Minister Yoshihiko Noda held talks in Beijing with Chinese Premier Wen Jiabao on Sunday. 
 The statement given to reporters gave no details of when Japan might seek to buy the bonds or on what scale. Japan had earlier indicated it is in talks to buy Chinese government debt to strengthen economic ties. Despite sometimes rancorous political ties between the two neighbors, Japan's economic fortunes are increasingly tied to China's economic growth and consumer demand. China has been Japan's biggest trading partner since 2009. In 2010, trade between the two nations grew by 22.3 percent compared to levels in 2009, reaching 26.5 trillion yen ($339.3 billion), according to the Japan Extern

-- Japan to enter dollar swap agreement with India
-- 
-- Sat Dec 24, 2011 9:35pm EST
-- http://www.reuters.com/article/2011/12/25/us-japan-india-swap-idUSTRE7BO00P20111225

 

 TOKYO  (Reuters) - The Japanese government is considering a dollar swap arrangement with India to provide emergency liquidity in case the European debt crisis reaches emerging economies, the Nikkei business newspaper said on Sunday. 
 The agreement would set the total swap arrangement at $10 billion, or 780 billion yen, the Nikkei said. Both countries are looking to sign off on the arrangement next Wednesday, when leaders meet at a bilateral summit, the paper said. The currency swaps are expected to support the Indian rupee as it continues to weaken against the greenback and Europe's sovereign debt crisis hits India's exports. The dollar-swap arrangement with India would follow a similar agreement with South Korea in October. (Reporting by  Mari Saito ; Editing by  Paul Tait )
-- Reversal of fortune
-- By  Emil

-- IMF's Lagarde warns global economy threatened
-- 
-- Sun Dec 25, 2011 7:49am EST
-- http://www.reuters.com/article/2011/12/25/us-france-imf-idUSTRE7BO07120111225

 

 PARIS  (Reuters) - The head of the International Monetary Fund said the world economy was in danger and urged Europeans to speak with one voice on a debt crisis that has rattled the global financial system. 
 In Nigeria last week, IMF Christine Lagarde said the IMF's 4 percent growth forecast for the world economy in 2012 could be revised downward, but gave no new figure. "The world economy is in a dangerous situation," she told France's Journal du Dimanche in an interview published on Sunday. The debt crisis, which continues into 2012 after a European Union summit on December 9 only temporarily calmed markets, "is a crisis of confidence in public debt and in the solidity of the financial system," she said. European leaders drafted a new treaty for deeper economic integration in the euro zone, but it is not certain tha

In [52]:
for article in articles:
    print(article)

We use this code above to open up our Reuters folder and read the files from our directory. The data being used here comes from this repository [financial news corpus](https://github.com/philipperemy/financial-news-dataset). It is pretty great. 

In [9]:
import nltk

In [10]:
from nltk.chunk import conlltags2tree, tree2conlltags

sentence = "Mark and John are working at Google."

for sent in nltk.sent_tokenize(sentence):
   for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
      if hasattr(chunk, 'label'):
         print(chunk.label(), ' '.join(c[0] for c in chunk))

PERSON Mark
PERSON John
ORGANIZATION Google


Above is some example code from nltk's NE chunker/tagger. It works quite well in our purposes for this PMI task.

Below is where I get into the meat of calculating the PMI. 

$$pmi(x,y) = log\frac{p(x,y)}{p(x)p(y)}$$

Usually we define p(x,y) as the probability of the intersection of two entities within some window. For the purposes of this experiment, I am defining windows as sentences. Therefore, the equation we arrive for calculating PMI at is:

$$pmi(x,y) = log\frac{count(x,y)_{D}}{count(x)_{D}count(y)_{D}}$$

Where $$D$$ is all of the documents in the Reuters corpus. $$x$$ and $$y$$ are occurrences of a polarity word (positive when calculating positive PMI, negative words when calculating negative PMI). 

Each article is looped through in order to build the overall counts of words in order to count PMIs.

Additionally, we store the counts of all words as they relate to organizations.

In [42]:
import math
import nltk

In [40]:
def num_words(sentences):
    l = 0
    pos_count = 0
    neg_count = 0
    for s in sentences:
        l += len(s)
        for word in nltk.word_tokenize(s):
            if word.upper() in mcd.pos_words:
                pos_count += 1
            elif word.upper() in mcd.neg_words:
                neg_count += 1
    return l, pos_count, neg_count

In [71]:
for article in articles:
    print(article)
    l, pos_count, neg_count = num_words(nltk.sent_tokenize(article['text']))
    print('{} positve, {} negative'.format(pos_count,neg_count))
    
    print('\n\n')
    sys.stdout.flush()

{'text': ' House of Representatives Democratic Whip Steny Hoyer said on Friday that budget negotiators were "very close" to a deal to avoid a shutdown of the federal government. \n\n "There\'s no deal yet unfortunately," Hoyer told NBC\'s Today show program as negotiators worked furiously to reach agreement. "I think we\'re very close. I think we\'ve come 70 percent of the way in terms of dollars. That\'s a long way to go in trying to reach compromise." (Reporting by  David Morgan ; Editing by  Vicki Allen )', 'date': ' Fri Apr 8, 2011 7:15am EDT\n', 'title': ' House Democrat says budget deal "very close"\n', 'author': ' \n', 'link': ' http://www.reuters.com/article/2011/04/08/us-usa-budget-hoyer-idUSTRE73725M20110408\n\n \n\n WASHINGTON  ('}
0 positve, 2 negative



{'text': " President Barack Obama and leaders in the U.S. Congress are racing against a midnight Friday deadline to reach a deal on government spending for the rest of this fiscal year, which ends on September 30. \n\n Wit

{'text': ' "Let me be crystal clear," Barclays\' star investment banker and new Chief Executive Bob Diamond told members of the British parliament in January. "We are going to be here in the United Kingdom, and this is the place that we want to succeed." \n\n It may be a reflection of how bankers still struggle to be taken at their word in the wake of the global financial crisis, but Diamond\'s statement convinced nobody. Rumors that Barclays and its bigger rival HSBC are ready to move their headquarters to Hong Kong or New York have flourished ever since. Speculation has intensified in the run-up to the April 11 publication of an interim report into the future of British financial services by the Independent Commission on Banking (ICB). The banks fear it will recommend costly reforms. A public admission by Diamond last week that he had "an obligation on behalf of shareholders" to look at alternative tax bases, and repeated warnings from senior city figures about the consequences of a 

{'text': ' The Justice Department approved Google Inc\'s purchase of ITA Software with stiff conditions on Friday, and left the door open to a larger probe into whether Google manipulates its search results to hurt rivals. \n\n The Justice Department blessed the $700 million deal with the ticketing software company. Google promised to license the software for five years, to continue to upgrade it, and to establish firewalls to protect ITA clients\' intellectual property. The ITA buy is part of an acquisition and hiring spree as Google aims to ensure its online services stay on top as Internet surfers go mobile and turn to services like the wildly popular Facebook. Now, U.S. antitrust regulators will turn to the question of whether to open a formal antitrust probe into allegations that Google, the world\'s No. 1 Internet search engine, manipulates search results, a source told Reuters on Friday. The Federal Trade Commission and Justice Department are both contemplating an investigation 

{'text': ' Taiwan smartphone maker HTC Corp said first-quarter profit almost tripled, beating forecasts, driven by strong demand for its handsets, particularly those running on the Android operating system \n\n The company, which has just overtaken erstwhile industry giant Nokia Oyj in terms of market capitalization, said on Friday that unaudited first-quarter net profit was T$14.83 billion ($511 million). Analysts had forecast a profit of T$12.99 billion. "That its first quarter would be above expectations was well foreseen, Q1 seasonality was better than expected," said Bonnie Chang, an analyst at Yuanta Securities in Hong Kong. "For the second quarter everyone is expecting revenue sequential growth in the high teens to 20 percent, shipments will be strong and average selling prices are holding up pretty well." Growing demand for phones running on Google\'s Android platform will help the smartphone market grow in 2011, boosting companies such as HTC and Samsung Electronics who are be

1 positve, 5 negative



{'text': ' Here\'s some advice for anyone hoping not to get caught in an insider trading web: Shut up. \n\n After the October 2009 arrest of Galleon Group hedge fund founder Raj Rajaratnam, who was wiretapped by the FBI, the world woke to a new era of prosecutorial tactics in insider trading probes. For those considering crossing the line, the changes may make insider trading seem a vocation not worth the risk. Emails and other forms of electronic communication can be traced. Destroying old-fashioned snail mail or hard drives will not work if the government gets a tip it is being done. And don\'t talk to people. There is no certainty that what is said will remain private. Indeed, recent phone recordings make some accused wrongdoers come across as Keystone Kriminals. "Insider trading today is a lot more difficult to get away with than it was 20 years ago," said Andrew Stoltmann, a principal at Stoltmann Law Offices PC in Chicago who has handled civil insider tra

...

{'text': ' JPMorgan Chase & Co Chief Executive Jamie Dimon on Wednesday lashed out at the government for a lawsuit alleging misdeeds at Bear Stearns, more than four years after JPMorgan was asked to rescue the teetering financial giant. \n Dimon said the company is still paying the price for doing the Federal Reserve "a favor" by buying Bear Stearns in early 2008. "I\'m going to say we\'ve lost $5 billion to $10 billion on various things related to Bear Stearns now. And yes, I put it in the unfair category," Dimon said, speaking at a Council on Foreign Relations event in Washington. Dimon\'s candid comments come one week after the New York State Attorney General filed a lawsuit against JPMorgan, alleging that Bear Stearns deceived investors buying mortgage-backed securities in 2006 and 2007. During a wide-ranging hour-long discussion that went from the "fiscal cliff" to the impact of regulations, Dimon bristled when a member of the audience asked him if he now regretted participating w

{'text': ' The automotive industry faces a tougher year in 2012 than it has done this year, Audi ( NSUG.DE ) Chief Executive Rupert Stadler told Reuters in an interview for release on Sunday. \n "But this should not be a reason to fall into pessimism," he said, adding that the automotive industry will continue to grow over the next 10 years thanks to the enormous appetite for mobility among people in developing economies. Stadler also denied speculation in the German media that he might be a successor for Audi parent Volkswagen\'s ( VOWG_p.DE ) CEO, Martin Winterkorn, when he steps down at the end of 2016. He said he expects China\'s automotive market to grow 8 to 9 percent, with its premium segment expanding 20 percent or even 30 percent, though he did not give a timeframe. Audi is the market leader in the luxury segment in China, well ahead of German rivals BMW ( BMWG.DE ) and Daimler AG\'s ( DAIGn.DE ) Mercedes. Stadler, who has set a goal of making Audi the world\'s top premium car

{'text': ' Japan and China agreed to start formal talks early next year on a free trade pact that would also include South Korea, Japanese Prime Minister Yoshihiko Noda said on Sunday after talks that showed the deepening bonds between Asia\'s two biggest economies. \n Japan also said it was looking to buy Chinese treasury debt, and the two governments agreed to enhance financial cooperation. "On a free trade agreement among Japan, China and South Korea, we\'ve made a substantial progress for an early start of negotiations," Noda told reporters after his meeting with Premier Wen Jiabao. China\'s central bank, the People\'s Bank of China, said on its website (www.pbc.gov.cn) that the two leaders agreed to strengthen bilateral financial market cooperation and "encourage the use of the renminbi and Japanese yen in international trade transactions between the two countries." The renminbi is another name for China\'s yuan currency. The trade talks announcement builds on an agreement between

{'text': " The Japanese government is considering a dollar swap arrangement with India to provide emergency liquidity in case the European debt crisis reaches emerging economies, the Nikkei business newspaper said on Sunday. \n The agreement would set the total swap arrangement at $10 billion, or 780 billion yen, the Nikkei said. Both countries are looking to sign off on the arrangement next Wednesday, when leaders meet at a bilateral summit, the paper said. The currency swaps are expected to support the Indian rupee as it continues to weaken against the greenback and Europe's sovereign debt crisis hits India's exports. The dollar-swap arrangement with India would follow a similar agreement with South Korea in October. (Reporting by  Mari Saito ; Editing by  Paul Tait )", 'date': ' Sat Dec 24, 2011 9:35pm EST\n', 'title': ' Japan to enter dollar swap agreement with India\n', 'author': ' \n', 'link': ' http://www.reuters.com/article/2011/12/25/us-japan-india-swap-idUSTRE7BO00P20111225\n

{'text': ' The head of the International Monetary Fund said the world economy was in danger and urged Europeans to speak with one voice on a debt crisis that has rattled the global financial system. \n In Nigeria last week, IMF Christine Lagarde said the IMF\'s 4 percent growth forecast for the world economy in 2012 could be revised downward, but gave no new figure. "The world economy is in a dangerous situation," she told France\'s Journal du Dimanche in an interview published on Sunday. The debt crisis, which continues into 2012 after a European Union summit on December 9 only temporarily calmed markets, "is a crisis of confidence in public debt and in the solidity of the financial system," she said. European leaders drafted a new treaty for deeper economic integration in the euro zone, but it is not certain that the accord will stem the debt crisis, which began in Greece in 2009, and now threatens France and even economic powerhouse Germany. "The December 9 summit wasn\'t detailed e

In [ ]:
def compute_PMI(class1, class2, int_c1c2, overall_count):
    return math.log((int_c1c2+1/overall_count)/((class1+1/overall_count)*(class2+1/overall_count)))
    # +1s added for smoothing

In [ ]:
import pandas as pd
import numpy as np

length = 0
overall_pos = 0
overall_neg = 0
overall_org = 0
intersection_pos = 0
intersection_neg = 0

pos_df = pd.DataFrame(0, index=[str(key) for (key,val) in mcd.pos_words.items()], columns=[])
neg_df = pd.DataFrame(0, index=[str(key) for (key,val) in mcd.neg_words.items()], columns=[])

These pandas dataframes will allow us to represent co-occurences of organizations and polarity words as a matrix. Initially, dataframes are initialized as 0 column matrices with row indexes as the words from the positive and negative financial polarity lists.

In [ ]:
for article in articles[:1000]:
    sentences = nltk.sent_tokenize(article.text)
    tmpL, tmp_pos, tmp_neg = num_words(sentences)
    length += tmpL
    overall_pos += tmp_pos
    overall_neg += tmp_neg
    for sent in sentences:
       org_count = 0
       pos_count = 0
       neg_count = 0
       org_list = []
       chunks = [chunk for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent)))]
       for chunk in chunks:
            if hasattr(chunk, 'label') and str(chunk.label()) == 'ORGANIZATION':
                #print(chunk.label())
                org_count += 1
                overall_org += 1
                org_list.append(str(chunk[0]).upper())
                if str(chunk[0]).upper() not in pos_df.columns:
                    print(str(chunk[0]).upper())
                    pos_df[str(chunk[0]).upper()] = np.zeros(len(pos_df.index))
                    neg_df[str(chunk[0]).upper()] = np.zeros(len(neg_df.index))
                
       tmp_org_count = org_count
       for chunk in chunks:
            #print(chunk[0])
            #print(mcd.pos_words)
            if str(chunk[0]).upper() in mcd.pos_words:
                tmp_org_list = org_list
                #print(chunk[0])
                pos_org_count = tmp_org_count
                while len(tmp_org_list) > 0:
                    pos_count += 1
                    pos_df.at[str(chunk[0]).upper(), tmp_org_list[0]] += 1
                    tmp_org_list.pop(0)
                    mcd.intersection_pos[str(chunk[0]).upper()] += 1
                mcd.pos_word_counts[str(chunk[0]).upper()] += 1
            elif str(chunk[0]).upper() in mcd.neg_words:
                #print(chunk[0])
                tmp_org_list = org_list  
                while(len(tmp_org_list) > 0):
                    neg_count += 1
                    neg_df.at[str(chunk[0]).upper(), tmp_org_list[0]] += 1
                    tmp_org_list.pop(0)
                    mcd.intersection_neg[str(chunk[0]).upper()] += 1
                mcd.neg_word_counts[str(chunk[0]).upper()] += 1
       intersection_pos += org_count if org_count < pos_count else pos_count
       intersection_neg += org_count if org_count < neg_count else neg_count
    #print(pos_count)
    #print(overall_org)
    #print(intersection)
    #print(l)
    
print(compute_PMI(overall_pos, overall_org, intersection_pos, l))
print(compute_PMI(overall_neg, overall_org, intersection_neg, l))
#print(mcd.pos_word_counts)
print(pos_df)


            
                
        
              #print(chunk.label(), ' '.join(c[0] for c in chunk))
    #create_co_occurrence_matrix(sentences)
    

%%latex

Here is the above algorithm:

\begin{enumerate}
    \item Initialize two empty Pandas dataframes, one for the positive words and one for the negative words.
    \item Loop through each article:
    \begin{enumerate}
        \item Tokenize each sentence in the article using NLTK.
            \begin{enumerate}
                \item Initialize an organization word count, a positive word count, a negative word count, and an empty list of orgs.
                \item Chunk the sentence using the NLTK NER chunker. Loop through each chunk and append the organization to the org list. If the organization is not present in the columns of the dataframe, insert a new column into the dataframe. 
                \item Loop through the chunks a second time. If the chunked word is in the positive (or negative, conversely) words dict, create a temporary organization list and increment the counts of the positive (negative) count (representing the number of co-occurrences in a sentence) and increment the index in the pandas dataframe corresponding to that positive or negative word.
            \end{enumerate}
    \end{enumerate}
\end{enumerate}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sorted_counts = sorted(mcd.pos_word_counts.items(), key=lambda kv: kv[1], reverse=True)
print(sorted_counts)
print(sorted_counts[50][0])
print(mcd.intersection_pos[sorted_counts[50][0]])
print(compute_PMI(sorted_counts[100][1], overall_org, mcd.intersection_pos[sorted_counts[100][0]], l))

sorted_counts[0:50]

plt.figure(figsize=(20, 3))  # width:20, height:3
# save the names and their respective scores separately
# reverse the tuples to go from most frequent to least frequent 
plt.bar(range(len(sorted_counts[0:20])), [val[1] for val in sorted_counts[0:20]], align='edge', width=.3)
plt.xticks(range(len(sorted_counts[0:20])), [val[0] for val in sorted_counts[:20]])
plt.xticks(rotation=70)
plt.show()


PMIs = [compute_PMI(count[1], overall_org, mcd.intersection_pos[count[0]], l) for count in sorted_counts[0:20]]

plt.figure(figsize=(20, 3))  # width:20, height:3
# save the names and their respective scores separately
# reverse the tuples to go from most frequent to least frequent 
plt.bar(range(len(sorted_counts[0:20])), PMIs, align='edge', width=.3)
plt.xticks(range(len(sorted_counts[0:20])), [val[0] for val in sorted_count[:20]])
plt.xticks(rotation=70)
plt.show()